In [ ]:
import math
import numpy as np
import json
import pandas as pd
import altair as alt

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.graph import graph_spinorama, graph_params_default, graph_freq
from spinorama.compute_normalize import normalize_mean, normalize_cea2034

In [ ]:
def parse_graph_freq_webplotdigitizer(filename):
    """ """
    # from 20Hz to 20kHz, log(2)~0.3                                                                                                                     
    ref_freq = np.logspace(1+math.log10(2), 4+math.log10(2), 500)
    #                                                                                                                                                    
    with open(filename, 'r') as f:
        # data are stored in a json file.                                                                                                                
        speaker_data = json.load(f)
        # store all results                                                                                                                              
        res = []
        for col in speaker_data['datasetColl']:
            data = col['data']
            # sort data                                                                                                                                  
            udata = [(data[d]['value'][0],
                      data[d]['value'][1])
                     for d in range(0, len(data))]
            sdata = sorted(udata, key=lambda a: a[0])
            #print(col['name'], len(sdata))                                                                                                              
            #print(sdata[0])                                                                                                                             
            # since sdata and freq_ref are both sorted, iterate over both                                                                                
            ref_p = 0
            for di in range(0, len(sdata)-1):
                d = sdata[di]
                dn = sdata[di+1]
                fr = d[0]
                db = d[1]
                frn = dn[0]
                dbn = dn[1]
                # remove possible errors                                                                                                                 
                if fr == frn:
                    continue
                # look for closest match                                                                                                                 
                while ref_freq[ref_p] <= fr:
                    if ref_p >= len(ref_freq)-1:
                        break
                    ref_p += 1
                # if ref_f is too large, skip                                                                                                            
                ref_f = ref_freq[ref_p]
                if ref_f > frn:
                    continue
                # linear interpolation                                                                                                                   
                ref_db = db+((dbn-db)*(ref_f-fr))/(frn-fr)
                #print('fr={:.2f} fr_ref={:.2f} fr_n={:.2f} db={:.1f} db_ref={:.1f} db_n={:.1f}'.format(fr, ref_f, frn, db, ref_db, dbn))                
                res.append([ref_f, ref_db, col['name']])

        # build dataframe                                                                                                                                
        freq = np.array([res[i][0] for i in range(0, len(res))]).astype(np.float)
        dB   = np.array([res[i][1] for i in range(0, len(res))]).astype(np.float)
        mrt  = [res[i][2] for i in range(0, len(res))]
        df = pd.DataFrame({'Freq': freq, 'dB': dB, 'Measurements': mrt})
        # print(df)                                                                                                                                      
        return 'CEA2034', df


In [ ]:
file = 'datas/Vendors/JBL/JBL 4367/JBL 4367.json'
title, dfs = parse_graph_freq_webplotdigitizer(file)
mean = normalize_mean(dfs)
dfs = normalize_cea2034(dfs, mean)

In [ ]:
params = graph_params_default.copy()
#params['ymin'] = -100
#params['ymax'] = +100
g = graph_spinorama(dfs, params)
dfs[dfs.Measurements == 'Sound Power DI']
g

In [ ]:
on = dfs[dfs.Measurements == 'On Axis'].rename(columns={'dB': 'ON'}).set_index('Freq')
lw = dfs[dfs.Measurements == 'Listening Window'].rename(columns={'dB': 'LW'}).set_index('Freq')
er = dfs[dfs.Measurements == 'Early Reflections'].rename(columns={'dB': 'ER'}).set_index('Freq')
sp = dfs[dfs.Measurements == 'Sound Power'].rename(columns={'dB': 'SP'}).set_index('Freq')

# align 2 by 2
align = on.align(lw, axis=0)
print(align[0].shape)
align = align[0].align(er, axis=0)
print(align[0].shape)
all_align = align[0].align(sp, axis=0)
print(all_align[0].shape)
# realigned with the largest frame
all_lw = all_align[0].align(lw, axis=0)
all_er = all_align[0].align(er, axis=0)
all_sp = all_align[0].align(sp, axis=0)
print(all_lw[1].shape, all_er[1].shape, all_sp[1].shape)
# extract right parts and interpolate
a_lw = all_lw[1].drop('Measurements', axis=1).interpolate()
a_er = all_er[1].drop('Measurements', axis=1).interpolate()
a_sp = all_sp[1].drop('Measurements', axis=1).interpolate()
print(a_lw.shape, a_er.shape, a_sp.shape)
# remove NaN numbers
# (a_lw.isna() | a_er.isna() | a_sp.isna() ).any()
res2 = pd.DataFrame({'Freq': a_lw.index, 
                     'Listening Window': a_lw.LW, 
                     'Early Reflections': a_er.ER, 
                     'Sound Power': a_sp.SP})
res = res2.dropna().reset_index(drop=True)


In [ ]:
from src.spinorama.load import graph_melt
source = graph_melt(res)
source.head()

In [ ]:
spin2 = graph_spinorama(source, graph_params_default)

In [ ]:
from src.spinorama.compute_cea2034 import estimated_inroom

lw = source[source.Measurements == 'Listening Window'].reset_index(drop=True)
er = source[source.Measurements == 'Early Reflections'].reset_index(drop=True)
sp = source[source.Measurements == 'Sound Power'].reset_index(drop=True)
eir = estimated_inroom(lw, er, sp)
graph_melt(eir).head()

In [ ]:
spin2 & graph_freq(graph_melt(eir), graph_params_default)